# **Preprocessing in NLP**

## 학습 목표
1. 정규 표현식(regular expression)을 정의하고 어떻게 쓰이는지 살펴본다.
2. 영어 데이터 전처리 방법들을 살펴보고 연습한다.
3. 한국어 데이터 전처리 방법들을 살펴보고 연습한다.

**Context**
1. 파이썬 re 라이브러리로 정규 표현식 써보기
2. NLTK 라이브러리로 영어 데이터 전처리하기
3. KoNLPy 패키지로 한국어 데이터 전처리하기

##0. 필요한 패키지 및 라이브러리 import 하기

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 41.4 MB/s eta 0:00:00


In [2]:
import re

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer, RegexpStemmer, WordNetLemmatizer
from konlpy.tag import Hannanum, Kkma, Komoran, Okt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## 1. 파이썬 re 라이브러리로 정규 표현식 써보기

```
💡 정규 표현식(regular expression)이란❓

특정한 규칙을 가진 문자열의 집합을 표현하는 데 사용하는 형식 언어이다.

Raw text는 불필요한 문자(character)들이 상당히 많은데,
사용자가 처리하고 싶은 string 패턴을 설정하면 주어진 데이터를 자동적으로 정제할 수 있게 된다.

파이썬의 re 라이브러리를 통해 정규 표현식에 대해 알아가고 사용법을 살펴보고자 한다.
```

참고: [위키백과](https://ko.wikipedia.org/wiki/%EC%A0%95%EA%B7%9C_%ED%91%9C%ED%98%84%EC%8B%9D)

### 1-1. 정규 표현식의 기초, 메타 문자들

```
💡 메타 문자란❓

원래 그 문자가 가진 뜻이 아닌 특별한 용도로 사용하는 문자를 뜻한다.

메타 문자들은 다음과 같다: . ^ $ * + ? { } [ ] \ | ( )
```
참고: https://wikidocs.net/4308

`[]` 문자 클래스
- `[]` 안에 들어가는 문자들 중 한 개의 문자와 매치한다.

In [3]:
# 매치의 여부를 프린트하는 간단한 함수
def check_match(match):
    if match:
        print("매치!")
    else:
        print("매치가 없음.")

In [4]:
pattern = re.compile("[abc]")     # 패턴 "[abc]"를 설정
match = pattern.search("awesome") # string "awesome"에 문자 "a", "b", 혹은 "c"가 존재하는지 확인

print(match)
check_match(match)

<re.Match object; span=(0, 1), match='a'>
매치!


> 💡 `re.search(pattern, string)` 함수로 컴파일 스텝을 건너뛸 수 있다.

In [5]:
re.search("[abc]", "aa")

<re.Match object; span=(0, 1), match='a'>

> 매치가 없으면 `None` 객체를 리턴한다.

In [6]:
match = re.search("[abc]", "dude")
print(match)
check_match(match)

None
매치가 없음.


> [ ] 안의 두 문자 사이에 하이픈(-)을 사용하면 두 문자 사이의 범위를 의미한다.
>
>예를 들어 [a-c]는 [abc]와 동일하고 [0-3]는 [0123]와 동일하다.
>
>다음은 하이픈(-)을 사용한 문자 클래스의 사용 예이다.
>
>- `[a-zA-Z]` : 알파벳 모두
>- `[0-9]` : 숫자

In [ ]:
match = re.search("[a-zA-Z0-9]", "!@%!$!#@0")   # 숫자나 알파벳이 해당 텍스트에 존재하는지 확인
check_match(match)

매치!


`.` 기호
- 임의의 한 개의 문자를 나타낸다.

In [ ]:
# "x"가 "a"와 "b" 사이에 존재하기에 매치된다.
match = re.search("a.b", "axb")
check_match(match)

매치!


In [ ]:
# "a"와 "b" 사이에 존재하는 문자가 없기에 매치되지 않는다.
match = re.search("a.b", "abc")
check_match(match)

매치가 없음.


`*` 기호
- 바로 앞 정규식이 0번 이상 반복될 경우를 나타낸다.

In [ ]:
# "a"가 0번 반복되기 때문에 매치
match = re.search("ca*t", "ct")
check_match(match)

매치!


In [ ]:
# "a"가 1번 반복되기 때문에 매치
match = re.search("ca*t", "cat")
check_match(match)

매치!


In [ ]:
# "a"가 여러 번 반복되기 때문에 매치
match = re.search("ca*t", "caaaaaaaaaat")
check_match(match)

매치!


> `*` 기호와 문자 클래스 `[]`를 같이 사용할 수 있다.

In [ ]:
# "x", "y", "z"에서 반복되기 때문에 매치
match = re.search("c[xyz]*t", "cxyxyzt")
check_match(match)

매치!


> 괄호 `()`를 통해 문자를 grouping 시켜줄 수 있다.

In [ ]:
# "xyz"가 1번 반복되기 때문에 매치
match = re.search("c(xyz)*t", "cxyzt")
check_match(match)

매치!


In [ ]:
# "xyz"가 반복되지 않기 때문에 매치가 안됨
match = re.search("c(xyz)*t", "cxyt")
check_match(match)

매치가 없음.


`+` 기호
- 바로 앞 정규식이 1번 이상 반복될 경우를 나타낸다.

In [ ]:
# "a"가 0번 반복되기 때문에 매치가 되지 않음
match = re.search("ca+t", "ct")
check_match(match)

매치가 없음.


In [ ]:
# "a"가 1번 반복되기 때문에 매치
match = re.search("ca+t", "cat")
check_match(match)

매치!


In [ ]:
# "a"가 1번 이상 여러 번 반복되기 때문에 매치
match = re.search("ca+t", "caaaaaaaaaat")
check_match(match)

매치!


`{m,n}`
- 반복 횟수를 m이상 n이하로 설정한다.
- 생략된 m은 0과 동일하며, 생략된 n은 무한대(2억 개 미만)의 의미를 갖는다.
- `{1,}`은 `+`와 동일하고, `{0,}`은 `*`와 동일하다.

In [ ]:
# "a"가 1번 반복되기 때문에 매치가 되지 않음
match = re.search("ca{2}t", "cat")
check_match(match)

매치가 없음.


In [ ]:
# "a"가 2번 반복되기 때문에 매치
match = re.search("ca{2}t", "caat")
check_match(match)

매치!


In [ ]:
# "a"가 5번 반복되기 때문에 매치
match = re.search("ca{2,5}t", "caaaaat")
check_match(match)

매치!


In [ ]:
# "a"가 6번 반복되기 때문에 매치가 되지 않음
match = re.search("ca{2,5}t", "caaaaaat")
check_match(match)

매치가 없음.


`?` 기호
- `{0, 1}`과 동일하다.
- 바로 앞 정규식이 있어도 되고 없어도 된다.

In [ ]:
# "a"가 0번 사용되어 매치
match = re.search("ca?t", "ct")
check_match(match)

매치!


In [ ]:
# "a"가 1번 사용되어 매치
match = re.search("ca?t", "cat")
check_match(match)

매치!


In [ ]:
# "a"가 2번 사용되어 매치가 되지 않음
match = re.search("ca?t", "caat")
check_match(match)

매치가 없음.


`|` 기호
- or과 동일한 의미로 사용된다. `A|B`라는 정규식이 있다면 `A` 또는 `B`라는 의미가 된다.

In [ ]:
# 텍스트에 "Good"이나 "NLP"가 있어서 매치
match = re.search("Good|NLP", "NLP is fun.")
check_match(match)

매치!


In [ ]:
# 텍스트에 "Good"이나 "NLP"가 있어서 매치
match = re.search("Good|NLP", "Good morning.")
check_match(match)

매치!


In [ ]:
# 텍스트에 "Good"이나 "NLP"가 있지 않아서 매치가 되지 않음
match = re.search("Good|NLP", "Happy birthday!")
check_match(match)

매치가 없음.


`^` 기호
- 문자열의 맨 처음과 일치할 경우를 나타낸다.

In [ ]:
# "NLP"로 시작되기 때문에 매치
match = re.search("^NLP", "NLP is fun.")
check_match(match)

매치!


In [ ]:
# "NLP"로 시작되지 않기 때문에 매치가 되지 않음
match = re.search("^NLP", "Is NLP fun?")
check_match(match)

매치가 없음.


`[^문자]` 기호
- `^` 기호가 `[]` 문자 클래스 안에 있으면 의미가 달라진다.
- `^` 기호 뒤에 붙은 문자들을 제외한 모든 문자를 매치한다.

In [ ]:
# "a", "b", "c"를 제외한 문자가 존재하지 않아 매치가 되지 않음
match = re.search("[^abc]", "ab")
check_match(match)

매치가 없음.


In [ ]:
# "d"가 "a", "b", "c"를 제외한 문자라 매치
match = re.search("[^abc]", "abcd")
check_match(match)

매치!


> 💡 알아두면 유용한 shortcut 표현들이다.
>
>- `\d` : 숫자(digit)와 매치, [0-9]와 동일
- `\D` : 숫자가 아닌 것과 매치, [^0-9]와 동일
- `\s` : whitespace 문자와 매치, [ \t\n\r\f\v]와 동일. 맨 앞의 빈 칸은 공백문자(space)를 의미.
- `\S` : whitespace 문자가 아닌 것과 매치, [^ \t\n\r\f\v]와 동일
- `\w` : 문자+숫자(alphanumeric)와 매치, [a-zA-Z0-9_]와 동일
- `\W` : 문자+숫자(alphanumeric)가 아닌 문자와 매치, [^a-zA-Z0-9_]와 동일

In [ ]:
print(re.search("\d", "42"))            # 숫자 4와 매치
print(re.search("\d", "abc"))           # 숫자가 존재하지 않아 매치가 되지 않음
print(re.search("\D", "1234"))          # 숫자가 아닌 것이 존재하지 않아 매치가 되지 않음
print(re.search("\D", "cat"))           # 숫자가 아닌 "c"와 매치
print(re.search("\s", "NLP is fun."))   # whitespace가 있어서 매치
print(re.search("\s", "NLP"))           # whitespace가 없어서 매치가 되지 않음
print(re.search("\S", "NLP"))           # whitespace가 없어서 매치
print(re.search("\S", "   "))           # whitespace가 아닌 것이 없어서 매치가 되지 않음
print(re.search("\w", "nlp1234"))       # alphanumeric 문자가 있어서 매치
print(re.search("\W", "nlp1234"))       # alphanumeric 문자가 없어서 매치가 되지 않음

<re.Match object; span=(0, 1), match='4'>
None
None
<re.Match object; span=(0, 1), match='c'>
<re.Match object; span=(3, 4), match=' '>
None
<re.Match object; span=(0, 1), match='N'>
None
<re.Match object; span=(0, 1), match='n'>
None


### 1-2. 정규식을 이용한 문자열 검색

`match()`
- 첫 문자열과 정규식이 매치되는지 조사한다.

In [ ]:
pattern = re.compile("[a-z]+")

print(pattern.match("nlp"))     # "nlp" 문자열은 정규식 [a-z]+에 부합
print(pattern.match("1 nlp"))   # 처음에 나오는 문자 1이 정규식 [a-z]+에 부합되지 않음

<re.Match object; span=(0, 3), match='nlp'>
None


`search()`
- 문자열 전체를 검색하여 정규식과 매치되는지 조사한다.

In [ ]:
pattern = re.compile("[a-z]+")

print(pattern.search("nlp"))     # "nlp" 문자열은 정규식 [a-z]+에 부합
print(pattern.search("1 nlp"))   # 처음에 나오는 문자는 1이지만 다음 문자열 "nlp"가 정규식 [a-z]+에 부합

<re.Match object; span=(0, 3), match='nlp'>
<re.Match object; span=(2, 5), match='nlp'>


`findall()`
- 정규식과 매치되는 모든 문자열을 리스트로 리턴한다.

In [ ]:
pattern = re.compile("[a-z]+")

result = pattern.findall("nlp is fun")
print(result)

['nlp', 'is', 'fun']


`finditer()`
- 정규식과 매치되는 모든 문자열을 반복 가능한 객체(iterable object)로 리턴한다.

In [ ]:
result = pattern.finditer("nlp is fun")
print(result)

for r in result:
    print(r)

<re.Match object; span=(0, 3), match='nlp'>
<re.Match object; span=(4, 6), match='is'>
<re.Match object; span=(7, 10), match='fun'>


## 2. NLTK 라이브러리로 영어 데이터 전처리하기

### 2-1. 구두점(punctuations) 제거

`re.sub(pattern, repl, string)`
- `string`에서 매치 되는 `pattern`을 `repl`로 바꾼다.

In [ ]:
# 구두점을 empty string('')으로 대체
print(re.sub("[!\"#$%&\'\(\)\[\]*+,-./:;<=>?@^+`{|}~\\\]", '', "NLP is fun!\"#$%&'()[]*+,-./:;<=>?@^+`{|}~\\"))

NLP is fun


### 2-2. Lowercasing

In [ ]:
text = "NLP is fun"
print(text.lower())

nlp is fun


### 2-3. 불용어(stopwords) 제거

```
💡 Stopwords란❓

의미가 많이 중요하지 않은 단어들을 나타낸다.

이런 단어들은 언어 모델 학습에 도움이 안된다고 판단되면 사용자가 제거할 수도 있다.
```

In [ ]:
# nltk 라이브러리에서 정의한 stopwords 리스트를 불러온다.
stopwords = nltk.corpus.stopwords.words('english')
print(f"Stopwords: {stopwords}")
print(f"Stopwords 갯수: {len(stopwords)}")

Stopwords: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so'

> Stopwords를 제거하기 위한 간단한 함수를 정의한다.

In [ ]:
def remove_stopwords(text):
    output = [word for word in text if word not in stopwords]
    return output

> 테스트 텍스트를 통해 잘 작동하는지 확인한다.

In [ ]:
test_text = "Stopwords do not carry essential meanings, so it's better to remove them during preprocessing."
tokenized_text = test_text.split()
print(remove_stopwords(tokenized_text))

['Stopwords', 'carry', 'essential', 'meanings,', 'better', 'remove', 'preprocessing.']


### 2-4. 어간 추출(Stemming)

```
💡 Stemming이란❓

단어의 어미를 자르는 작업으로써, 섬세한 작업이 아니기 때문에 사전에 존재하지 않는 단어일 수도 있다.

Stemming하는 알고리즘은 다양한데 NLTK 라이브러리에서는 총 네 가지를 사용할 수 있다.
1. Porter Stemmer          # basic
2. Snowball Stemmer        # porter version 2
3. Lancaster Stemmer       # excessive stemming
4. Regexp Stemmer          # custom rules

자세한 알고리즘 디테일은 documentation을 참고하기 바란다.

```
Documentation: https://www.nltk.org/api/nltk.stem.html

In [ ]:
# nltk 라이브러리의 Stemmer들을 불러온다.
porter = PorterStemmer()
snowball = SnowballStemmer(language="english")
lancaster = LancasterStemmer()
regexp = RegexpStemmer('ing$|s$|e$|able$', min=4)   # 정규표현식에 매치되는 길이가 4 이상인 string만 stemming을 진행

In [ ]:
def porter_stem(words):
    stemmed_words = [porter.stem(word) for word in words]
    return stemmed_words

def snowball_stem(words):
    stemmed_words = [snowball.stem(word) for word in words]
    return stemmed_words

def lancaster_stem(words):
    stemmed_words = [lancaster.stem(word) for word in words]
    return stemmed_words

def regexp_stem(words):
    stemmed_words = [regexp.stem(word) for word in words]
    return stemmed_words

In [ ]:
words = ["am", "is", "are", "founded", "glasses", "finishing", "crazy", "assignment"]

print('어간 추출 전 :', words)
print('어간 추출 후 (Porter):', porter_stem(words))
print('어간 추출 후 (Snowball):', snowball_stem(words))
print('어간 추출 후 (Lancaster):', lancaster_stem(words))
print('어간 추출 후 (Regexp):', regexp_stem(words))

어간 추출 전 : ['am', 'is', 'are', 'founded', 'glasses', 'finishing', 'crazy', 'assignment']
어간 추출 후 (Porter): ['am', 'is', 'are', 'found', 'glass', 'finish', 'crazi', 'assign']
어간 추출 후 (Snowball): ['am', 'is', 'are', 'found', 'glass', 'finish', 'crazi', 'assign']
어간 추출 후 (Lancaster): ['am', 'is', 'ar', 'found', 'glass', 'fin', 'crazy', 'assign']
어간 추출 후 (Regexp): ['am', 'is', 'are', 'founded', 'glasse', 'finish', 'crazy', 'assignment']


### 2-5. 표제어 추출(Lemmatization)

```
💡 Lemmatization이란❓

단어들이 다른 형태를 가지더라도 그 뿌리 단어를 찾아가서 단어의 개수를 줄일 수 있는지 판단하는 것이다.

예를 들어 am, is, are은 서로 다른 스펠링이지만 그 뿌리 단어는 be라고 볼 수 있다. 이때, 이 단어들의 표제어는 be다.
```

In [ ]:
lemmatizer = WordNetLemmatizer()

def lemmatize(words):
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return lemmatized_words

In [ ]:
words = ["am", "is", "are", "founded", "glasses", "finishing", "crazy", "assignment"]

print('표제어 추출 전 :', words)
print('표제어 추출 후 :', lemmatize(words))

표제어 추출 전 : ['am', 'is', 'are', 'founded', 'glasses', 'finishing', 'crazy', 'assignment']
표제어 추출 후 : ['am', 'is', 'are', 'founded', 'glass', 'finishing', 'crazy', 'assignment']


> 💡 표제어 추출은 어간 추출과는 달리 단어의 형태가 적절히 보존되는 양상을 보인다. 하지만 그럼에도 위의 결과에서는 표제어를 제대로 추출하지 못하고 있다. 이는 표제어 추출기(lemmatizer)가 본래 단어의 품사 정보를 알아야만 정확한 결과를 얻을 수 있기 때문이다.
>
> WordNetLemmatizer는 입력으로 단어의 품사를 알려줄 수 있다.
> - "n" for nouns, "v" for verbs, "a" for adjectives, "r" for adverbs and "s" for satellite adjectives

참고: https://wikidocs.net/21707

In [ ]:
lemmatizer.lemmatize("am", "v")

'be'

In [ ]:
lemmatizer.lemmatize("is", "v")

'be'

In [ ]:
lemmatizer.lemmatize("finishing", "v")

'finish'

## 3 KoNLPy 패키지로 한국어 데이터 전처리하기

> 한국어 데이터는 영어 데이터와 형태가 많이 다르기 때문에 전처리 하는 방식도 많이 다르다.
>
> 소문자, 대문자 개념이 없을뿐더러, 불용어도 한국어 문화에 맞춰진 단어들로 구성될 것이다.
>
> 접두사나 접미사들도 영어에 비해 다른 용도로 사용된다. (예: '맨'발, '맨'주먹, 선생'님', 교수'님')

> 다음은 ranks.nl 사이트에서 가져온 한국어 stopwords 리스트이다. 참고만 하길 바란다.


아
휴
아이구
아이쿠
아이고
어
나
우리
저희
따라
의해
을
를
에
의
가
으로
로
에게
뿐이다
의거하여
근거하여
입각하여
기준으로
예하면
예를 들면
예를 들자면
저
소인
소생
저희
지말고
하지마
하지마라
다른
물론
또한
그리고
비길수 없다
해서는 안된다
뿐만 아니라
만이 아니다
만은 아니다
막론하고
관계없이
그치지 않다
그러나
그런데
하지만
든간에
논하지 않다
따지지 않다
설사
비록
더라도
아니면
만 못하다
하는 편이 낫다
불문하고
향하여
향해서
향하다
쪽으로
틈타
이용하여
타다
오르다
제외하고
이 외에
이 밖에
하여야
비로소
한다면 몰라도
외에도
이곳
여기
부터
기점으로
따라서
할 생각이다
하려고하다
이리하여
그리하여
그렇게 함으로써
하지만
일때
할때
앞에서
중에서
보는데서
으로써
로써
까지
해야한다
일것이다
반드시
할줄알다
할수있다
할수있어
임에 틀림없다
한다면
등
등등
제
겨우
단지
다만
할뿐
딩동
댕그
대해서
대하여
대하면
훨씬
얼마나
얼마만큼
얼마큼
남짓
여
얼마간
약간
다소
좀
조금
다수
몇
얼마
지만
하물며
또한
그러나
그렇지만
하지만
이외에도
대해 말하자면
뿐이다
다음에
반대로
반대로 말하자면
이와 반대로
바꾸어서 말하면
바꾸어서 한다면
만약
그렇지않으면
까악
툭
딱
삐걱거리다
보드득
비걱거리다
꽈당
응당
해야한다
에 가서
각
각각
여러분
각종
각자
제각기
하도록하다
와
과
그러므로
그래서
고로
한 까닭에
하기 때문에
거니와
이지만
대하여
관하여
관한
과연
실로
아니나다를가
생각한대로
진짜로
한적이있다
하곤하였다
하
하하
허허
아하
거바
와
오
왜
어째서
무엇때문에
어찌
하겠는가
무슨
어디
어느곳
더군다나
하물며
더욱이는
어느때
언제
야
이봐
어이
여보시오
흐흐
흥
휴
헉헉
헐떡헐떡
영차
여차
어기여차
끙끙
아야
앗
아야
콸콸
졸졸
좍좍
뚝뚝
주룩주룩
솨
우르르
그래도
또
그리고
바꾸어말하면
바꾸어말하자면
혹은
혹시
답다
및
그에 따르는
때가 되어
즉
지든지
설령
가령
하더라도
할지라도
일지라도
지든지
몇
거의
하마터면
인젠
이젠
된바에야
된이상
만큼	어찌됏든
그위에
게다가
점에서 보아
비추어 보아
고려하면
하게될것이다
일것이다
비교적
좀
보다더
비하면
시키다
하게하다
할만하다
의해서
연이서
이어서
잇따라
뒤따라
뒤이어
결국
의지하여
기대여
통하여
자마자
더욱더
불구하고
얼마든지
마음대로
주저하지 않고
곧
즉시
바로
당장
하자마자
밖에 안된다
하면된다
그래
그렇지
요컨대
다시 말하자면
바꿔 말하면
즉
구체적으로
말하자면
시작하여
시초에
이상
허
헉
허걱
바와같이
해도좋다
해도된다
게다가
더구나
하물며
와르르
팍
퍽
펄렁
동안
이래
하고있었다
이었다
에서
로부터
까지
예하면
했어요
해요
함께
같이
더불어
마저
마저도
양자
모두
습니다
가까스로
하려고하다
즈음하여
다른
다른 방면으로
해봐요
습니까
했어요
말할것도 없고
무릎쓰고
개의치않고
하는것만 못하다
하는것이 낫다
매
매번
들
모
어느것
어느
로써
갖고말하자면
어디
어느쪽
어느것
어느해
어느 년도
라 해도
언젠가
어떤것
어느것
저기
저쪽
저것
그때
그럼
그러면
요만한걸
그래
그때
저것만큼
그저
이르기까지
할 줄 안다
할 힘이 있다
너
너희
당신
어찌
설마
차라리
할지언정
할지라도
할망정
할지언정
구토하다
게우다
토하다
메쓰겁다
옆사람
퉤
쳇
의거하여
근거하여
의해
따라
힘입어
그
다음
버금
두번째로
기타
첫번째로
나머지는
그중에서
견지에서
형식으로 쓰여
입장에서
위해서
단지
의해되다
하도록시키다
뿐만아니라
반대로
전후
전자
앞의것
잠시
잠깐
하면서
그렇지만
다음에
그러한즉
그런즉
남들
아무거나
어찌하든지
같다
비슷하다
예컨대
이럴정도로
어떻게
만약
만일
위에서 서술한바와같이
인 듯하다
하지 않는다면
만약에
무엇
무슨
어느
어떤
아래윗
조차
한데
그럼에도 불구하고
여전히
심지어
까지도
조차도
하지 않도록
않기 위하여
때
시각
무렵
시간
동안
어때
어떠한
하여금
네
예
우선
누구
누가 알겠는가
아무도
줄은모른다
줄은 몰랏다
하는 김에
겸사겸사
하는바
그런 까닭에
한 이유는
그러니
그러니까
때문에
그
너희
그들
너희들
타인
것
것들
너
위하여
공동으로
동시에
하기 위하여
어찌하여
무엇때문에
붕붕
윙윙
나
우리
엉엉
휘익
윙윙
오호
아하
어쨋든
만 못하다	하기보다는
차라리
하는 편이 낫다
흐흐
놀라다
상대적으로 말하자면
마치
아니라면
쉿
그렇지 않으면
그렇지 않다면
안 그러면
아니었다면
하든지
아니면
이라면
좋아
알았어
하는것도
그만이다
어쩔수 없다
하나
일
일반적으로
일단
한켠으로는
오자마자
이렇게되면
이와같다면
전부
한마디
한항목
근거로
하기에
아울러
하지 않도록
않기 위해서
이르기까지
이 되다
로 인하여
까닭으로
이유만으로
이로 인하여
그래서
이 때문에
그러므로
그런 까닭에
알 수 있다
결론을 낼 수 있다
으로 인하여
있다
어떤것
관계가 있다
관련이 있다
연관되다
어떤것들
에 대해
이리하여
그리하여
여부
하기보다는
하느니
하면 할수록
운운
이러이러하다
하구나
하도다
다시말하면
다음으로
에 있다
에 달려 있다
우리
우리들
오히려
하기는한데
어떻게
어떻해
어찌됏어
어때
어째서
본대로
자
이
이쪽
여기
이것
이번
이렇게말하자면
이런
이러한
이와 같은
요만큼
요만한 것
얼마 안 되는 것
이만큼
이 정도의
이렇게 많은 것
이와 같다
이때
이렇구나
것과 같이
끼익
삐걱
따위
와 같은 사람들
부류의 사람들
왜냐하면
중의하나
오직
오로지
에 한하다
하기만 하면
도착하다
까지 미치다
도달하다
정도에 이르다
할 지경이다
결과에 이르다
관해서는
여러분
하고 있다
한 후
혼자
자기
자기집
자신
우에 종합한것과같이
총적으로 보면
총적으로 말하면
총적으로
대로 하다
으로서
참
그만이다
할 따름이다
쿵
탕탕
쾅쾅
둥둥
봐
봐라
아이야
아니
와아
응
아이
참나
년
월
일
령
영
일
이
삼
사
오
육
륙
칠
팔
구
이천육
이천칠
이천팔
이천구
하나
둘
셋
넷
다섯
여섯
일곱
여덟
아홉
령
영

출처: https://www.ranks.nl/stopwords/korean

> 한국어를 알맞게 전처리 하기 위해서 **형태소 분석기**를 사용한다.
>
> 형태소 분석기는 형태소(morpheme)을 비롯하여, 어근, 접두사/접미사, 품사(POS, part-of-speech) 등 다양한 언어적 속성의 구조를 파악한다.

```
Python의 KoNLPy 패키지를 활용하여 한국어 텍스트의 유용한 특성을 추출할 수 있다.

KoNLPy에는 다섯 가지 분석기를 사용할 수 있다.
1. Hannanum    # KAIST 개발
2. Kkma        # 서울대 개발
3. Komoran     # Shineware 개발
4. Mecab       # 교토대/은전한닢 프로젝트 개발
5. Okt         # 유호현(트위터) 개발

각 분석기마다 전처리 하는 방식이 다르다. 일반적으로 일상 대화 데이터 같은 경우 Okt(전 트위터) 분석기를 사용하고,
나머지 분석기들은 사용할 데이터에 적용하여 비교 후 가장 알맞은 것으로 선택한다.

Mecab은 추가 설치가 필요하기에 본 실습에서는 사용하지 않는다.
```

참고: https://konlpy.org/en/latest/api/konlpy.tag/

In [ ]:
# 각 분석기 선언
text = '환영합니다! 자연어 처리 수업은 재미있게 듣고 계신가요?'
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

> 각 분석기마다 세 가지 함수를 사용할 수 있다.
> - `morphs` : 형태소 추출
> - `nouns` : 명사 추출
> - `pos` : 품사(part-of-speech) 추출
>
> 필요에 따라 알맞은 함수를 사용하면 된다.

In [ ]:
print("형태소:", hannanum.morphs(text))
print("명사:", hannanum.nouns(text))
print("품사:", hannanum.pos(text))

형태소: ['환영', '하', 'ㅂ니다', '!', '자연어', '처리', '수업', '은', '재미있', '게', '듣', '고', '계시', 'ㄴ가', '요', '?']
명사: ['환영', '자연어', '처리', '수업']
품사: [('환영', 'N'), ('하', 'X'), ('ㅂ니다', 'E'), ('!', 'S'), ('자연어', 'N'), ('처리', 'N'), ('수업', 'N'), ('은', 'J'), ('재미있', 'P'), ('게', 'E'), ('듣', 'P'), ('고', 'E'), ('계시', 'P'), ('ㄴ가', 'E'), ('요', 'J'), ('?', 'S')]


In [ ]:
print("형태소:", kkma.morphs(text))
print("명사:", kkma.nouns(text))
print("품사:", kkma.pos(text))

형태소: ['환영', '하', 'ㅂ니다', '!', '자연어', '처리', '수업', '은', '재미있', '게', '듣', '고', '계시', 'ㄴ가요', '?']
명사: ['환영', '자연어', '처리', '수업']
품사: [('환영', 'NNG'), ('하', 'XSV'), ('ㅂ니다', 'EFN'), ('!', 'SF'), ('자연어', 'NNG'), ('처리', 'NNG'), ('수업', 'NNG'), ('은', 'JX'), ('재미있', 'VA'), ('게', 'ECD'), ('듣', 'VV'), ('고', 'ECE'), ('계시', 'VXA'), ('ㄴ가요', 'EFQ'), ('?', 'SF')]


In [ ]:
print("형태소:", komoran.morphs(text))
print("명사:", komoran.nouns(text))
print("품사:", komoran.pos(text))

형태소: ['환영', '하', 'ㅂ니다', '!', '자연어', '처리', '수업', '은', '재미있', '게', '듣', '고', '계시', 'ㄴ가요', '?']
명사: ['환영', '자연어', '처리', '수업']
품사: [('환영', 'NNG'), ('하', 'XSV'), ('ㅂ니다', 'EF'), ('!', 'SF'), ('자연어', 'NNP'), ('처리', 'NNG'), ('수업', 'NNG'), ('은', 'JX'), ('재미있', 'VA'), ('게', 'EC'), ('듣', 'VV'), ('고', 'EC'), ('계시', 'VX'), ('ㄴ가요', 'EF'), ('?', 'SF')]


In [ ]:
print("형태소:", okt.morphs(text))
print("명사:", okt.nouns(text))
print("품사:", okt.pos(text))

형태소: ['환영', '합니다', '!', '자연어', '처리', '수업', '은', '재미있게', '듣고', '계신가요', '?']
명사: ['환영', '자연어', '처리', '수업']
품사: [('환영', 'Noun'), ('합니다', 'Verb'), ('!', 'Punctuation'), ('자연어', 'Noun'), ('처리', 'Noun'), ('수업', 'Noun'), ('은', 'Josa'), ('재미있게', 'Adjective'), ('듣고', 'Verb'), ('계신가요', 'Adjective'), ('?', 'Punctuation')]


> Okt 클래스에는 일상 대화 데이터를 정제하기 위한 `norm`과 `stem` 파라미터를 줄 수 있다.

In [ ]:
print(okt.pos(u'이것도 되나욬ㅋㅋ'))
print(okt.pos(u'이것도 되나욬ㅋㅋ', norm=True))
print(okt.pos(u'이것도 되나욬ㅋㅋ', norm=True, stem=True))

[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나욬', 'Noun'), ('ㅋㅋ', 'KoreanParticle')]
[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되나요', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]
[('이', 'Determiner'), ('것', 'Noun'), ('도', 'Josa'), ('되다', 'Verb'), ('ㅋㅋ', 'KoreanParticle')]
